# Weights and Biases

Notebook with code from [https://theaisummer.com/weights-and-biases-tutorial/](https://theaisummer.com/weights-and-biases-tutorial/)

In [1]:
# define example net
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [ ]:
import wandb

In [ ]:
transform = transforms.Compose(
                [transforms.ToTensor(), 
                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

class Net(nn.Module):

    def __init__(self, fc_layer_size=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, fc_layer_size)
        self.fc3 = nn.Linear(fc_layer_size, 10)

    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# set up device
if torch.cuda.is_available():
 dev = "cuda:0"
else:
 dev = "cpu"
print(dev)
device = torch.device(dev)

In [ ]:
# init wandb
run = wandb.init(project='test')

# set config
config = wandb.config
config.learning_rate = 0.01
config.epochs = 10

# instatiate net
net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=config.learning_rate)

# train
for epoch in range(config.epochs):
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            wandb.log({'epoch': epoch+1, 'loss': running_loss/2000})
            wandb.watch(net, criterion, log="all")
            running_loss = 0.0

print('Finished Training')

# Visualization

uses the train model, to visualize the training process and send it to W&B

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

columns=['image','label']
data = []

for i, batch in enumerate(trainloader, 0):
    inputs, labels = batch[0], batch[1]
    for j, image in enumerate(inputs,0):
        data.append([wandb.Image(image),classes[labels[j].item()]])
    break

table= wandb.Table(data=data, columns=columns)
run.log({"cifar10_images": table})

# Artifacts

versioncontrol on your data by registering in as an artifact

In [ ]:
cifar10_artifact = wandb.Artifact("cifar10", type="dataset")
file_path = './data/cifar-10-batches-py'
cifar10_artifact.add_dir(file_path)
run.log_artifact(cifar10_artifact)

# Sweeps

like doing a grid search on your runs

either a range of values or different values

"a sweep that tries all combinations to find the best hyperparameters for your model."

In [ ]:
import math
sweep_config = {
                'method': 'random',
                'metric': {'goal': 'minimize', 'name': 'loss'},
                'parameters': {
                    'batch_size': {
                        'distribution': 'q_log_uniform_values',
                        'max': 256,
                        'min': 32 },
                    'epochs': {'value': 5},
                    'fc_layer_size': {'values': [128, 256, 512]},
                    'learning_rate': {'distribution': 'uniform',
                                      'max': 0.1,
                                      'min': 0},
                    'optimizer': {'values': ['adam', 'sgd']}
                }
 }

sweep_id = wandb.sweep(sweep_config, project="test")

In [ ]:
def train(config=None):

    with wandb.init(project='test', entity='serkar', config=config):
        config = wandb.config

        transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

        trainloader = torch.utils.data.DataLoader(trainset, batch_size=config.batch_size, shuffle=True, num_workers=2)

        testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

        net = Net(config.fc_layer_size)
        net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(net.parameters(), lr=config.learning_rate)

        if config.optimizer == "sgd":
            optimizer = torch.optim.SGD(net.parameters(), lr=config.learning_rate, momentum=0.9)

        elif optimizer == "adam":
            optimizer = torch.optim.Adam(net.parameters(), lr=config.learning_rate)

        wandb.watch(net, criterion, log="all")

        for epoch in range(config.epochs):  # loop over the dataset multiple times

            running_loss = 0.0

            for i, data in enumerate(trainloader, 0):

                inputs, labels = data[0].to(device), data[1].to(device)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / len(trainloader)))

            wandb.log({'epoch': epoch + 1, 'loss': running_loss / len(trainloader)})

        print('Finished Training')

In [ ]:
# run sweep
wandb.agent(sweep_id, function=train, count=5)